Lambda School Data Science

*Unit 4, Sprint 1, Module 4*

---

# Topic Modeling (Prepare)

# Latent Dirchilet Allocation (LDA) Models (Prepare)
<a id="#p1"></a>

## Overview
LDA is a "generative probabilistic model". 

Let's play with a modoel available [here](https://lettier.com/projects/lda-topic-modeling/)

## Follow Along

## Challenge 

# Estimating LDA Models with Gensim (Learn)
<a id="#p1"></a>

## Overview
### A Litterary Introduction: *Jane Austen V. Charlotte Bronte*
Despite being born nearly forty years apart, modern fans often pit Jane Austen & Charlotte Bronte against one another in a battle for litterary  supremacy. The battle centers around the topics of education for women, courting, and marriage. The authors' similiar backgrounds naturally draw comparisons, but the modern fascination is probably due to novelility of British women publishing novels during the early 19th century. 

Can we help close a litterary battle for supremacy and simply acknowledge that the authors addressed different topics and deserve to be acknowledged as excellent authors each in their own right?

We're going to apply Latent Dirichlet Allocation a machine learning alogrithm for topic modeling to each of the author's novels to compare the distribution of topics in their novels.

In [ ]:
import numpy as np
import gensim
import os
import re

from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim import corpora

from gensim.models.ldamulticore import LdaMulticore

import pandas as pd

In [ ]:
# def tokenize(text):
#     return [token for token in simple_preprocess(text) if token not in STOPWORDS]

In [ ]:
def tokenize(text):
    tokens = []
    for token in simple_preprocess(text):
        if token not in STOPWORDS and len(token) > 3:
            tokens.append(token)
    return tokens

### Novel Data
I grabbed the novel data pre-split into a bunch of smaller chuncks

In [ ]:
path = './data/austen-brontë-split'

In [ ]:
import os

def gather_data(path_to_data): 
    data = []
    for f in os.listdir(path):
        if os.path.isdir(f) == False:
            if f[-3:] == 'txt':
                with open(os.path.join(path,f)) as t:
                    text = t.read().strip('\n')
                    data.append(tokenize(str(text)))       
    return data

In [ ]:
tokens = gather_data(path)

In [ ]:
tokens[0][0:10]

In [ ]:
len(tokens)

In [ ]:
df = pd.DataFrame(index=titles, data={'tokens':tokens})

In [ ]:
df['tokens'] = gather_data(path)

In [ ]:
"this is a sample string with a \n newline character".replace('\n', '')

In [ ]:
df['tokens'].head()

## Follow Along

### Text Preprocessing
**Challenge**: update the function `tokenize` with any technique you have learned so far this week. 

In [ ]:
import os

In [ ]:
titles = [t[:-4] for t in os.listdir(path)]

In [ ]:
titles[:5]

In [ ]:
len(titles)

In [ ]:
STOPWORDS = set(STOPWORDS).union(set(['said', 'mr', 'mrs']))

def tokenize(text):
    return [token for token in simple_preprocess(text) if token not in STOPWORDS]

In [ ]:
tokenize("Hello World! This a test of the tokenization method")

### Author DataFrame


In [103]:
df = pd.DataFrame(index=titles, data={'tokens':tokens})

In [104]:
df.head()

,tokens
Austen_Emma0000,"[emma, jane, austen, volume, chapter, emma, wo..."
Austen_Emma0001,"[taylor, wish, pity, weston, thought, agree, p..."
Austen_Emma0002,"[behaved, charmingly, body, punctual, body, be..."
Austen_Emma0003,"[native, highbury, born, respectable, family, ..."
Austen_Emma0004,"[mention, handsome, letter, weston, received, ..."


In [ ]:
df['author'] = df.reset_index()['index'].apply(lambda x: x.split('_')[0]).tolist()
df['book'] = df.reset_index()['index'].apply(lambda x: x.split('_')[1][:-4]).tolist()
df['section'] = df.reset_index()['index'].apply(lambda x: x[-4:]).tolist()
df['section'] = df['section'].astype('int')

In [ ]:
df['author'] = df['author'].map({'Austen':1, 'CBronte':0})

In [ ]:
df.author.value_counts()

### Streaming Documents
Here we use a new pythonic thingy: the `yield` statement in our fucntion. This allows us to iterate over a bunch of documents without actually reading them into memory. You can see how we use this fucntion later on. 

In [ ]:
# overkill function but it prevents overloading from gathering all that data in a list

def doc_stream(path):
    for f in os.listdir(path):
        if os.path.isdir(f) == False:
            if f[-3:] == 'txt':
                with open(os.path.join(path,f)) as t:
                    text = t.read().strip('\n')
                    tokens = tokenize(str(text))
                yield tokens

In [ ]:
# Call that stream-e-doo function on our path...

streaming_data = doc_stream(path)

In [ ]:
# If I call the streamer-thingy...

streaming_data
# It's saying "yes, this is a streaming object.", but can't really see it...

In [ ]:
# But... if I use the next command:

next(streaming_data)

# It looks at the next document. Run it again, it looks at the next one, etc.

### Gensim LDA Topic Modeling

In [ ]:
# Gensim works WILL work with a generator (scikit learn will not)




In [ ]:
# A Dictionary Representation of all the words in our corpus

# AKA "Bag of Words" -- (no ... you're a bag of words)

id2word = corpora.Dictionary(doc_stream(path))

In [ ]:
# Can look up the id of a word in the dic
id2word.token2id['chip']

In [ ]:
# Other way around: look up word with given id

id2word[69]

In [ ]:
# Looks up the word and frequency of each in the "sentence" given

id2word.doc2bow(tokenize("This is a sample message Darcy England England England"))

# returns a list of tuples

In [ ]:
import sys
sys.getsizeof(id2word)

In [ ]:
len(id2word.keys())

In [ ]:
# Let's remove extreme values from the dataset

# This is a way of tuning/trimming the words we use

id2word.filter_extremes(no_below=5, no_above=0.9)

In [ ]:
# Look at the number of words left after trimming

len(id2word.keys())

In [ ]:
# a bag of words(bow) representation of our corpus
# Note: we haven't actually read any text into memory here

# 

corpus = [id2word.doc2bow(text) for text in doc_stream(path)]

In [106]:
# look at the first ten words in the first document of our corpus

corpus[4][:10]

[(0, 2),
 (2, 1),
 (21, 2),
 (23, 3),
 (26, 1),
 (29, 2),
 (31, 2),
 (32, 1),
 (36, 1),
 (37, 1)]

In [ ]:
lda = LdaMulticore(corpus=corpus,
                   id2word=id2word,
                   random_state=723812,
                   num_topics = 15,
                   passes=10,
                   workers=4
                  )

In [ ]:
# Oooooo "topics"

lda.print_topics()

In [ ]:
# RegEx away all the extra silliness from the above output

words = [re.findall(r'"([^"]*)"',t[1]) for t in lda.print_topics()]

In [ ]:
topics = [' '.join(t[0:5]) for t in words]

In [ ]:
# Pretty print-job of topics without all the silly extra stuff

for id, t in enumerate(topics): 
    print(f"------ Topic {id} ------")
    print(t, end="\n\n")

## Challenge 

You will apply an LDA model to a customer review dataset to practice the fitting and estimation of LDA. 

# Interpret LDA Results (Learn)
<a id="#p3"></a>

## Overview

## Follow Along

### Topic Distance Visualization

In [ ]:
import pyLDAvis.gensim

pyLDAvis.enable_notebook()

In [ ]:
# This is going to become our visualizer 

pyLDAvis.gensim.prepare(lda, corpus, id2word)

### Overall Model / Documents

In [ ]:
lda[corpus[0]]

In [ ]:
distro = [lda[d] for d in corpus]

In [ ]:
distro[6]

In [ ]:
# Jon-Cody Special: takes the distrobutions and puts them in "toodense" format 
#                   (with bunch zeros...) See df.head() below

distro = [lda[d] for d in corpus]

def update(doc):
        d_dist = {k:0 for k in range(0,15)}
        for t in doc:
            d_dist[t[0]] = t[1]
        return d_dist
    
new_distro = [update(d) for d in distro]

In [ ]:
# turns jon_cody output into a pandas df

df = pd.DataFrame.from_records(new_distro, index=titles)
df.columns = topics
df['author'] = df.reset_index()['index'].apply(lambda x: x.split('_')[0]).tolist()

In [ ]:
df.head()

In [ ]:
# Very Nice!!! Check that shit out.

df.groupby('author').mean()

## Challenge
### *Can we see if one of the authors focus more on men than women?*

*  Use Spacy for text prepocessing
*  Extract the Named Entities from the documents using Spacy (command is fairly straight forward)
*  Create unique list of names from the authors (you'll find that there are different types of named entities not all people)
*  Label the names with genders (can you this by hand or you use the US census name lists)
*  Customize your processing to replace the proper name with your gender from the previous step's lookup table
*  Then follow the rest of the LDA flow


# Selecting the Number of Topics (Learn)
<a id="#p4"></a>

## Overview

## Follow Along

In [ ]:
from gensim.models.coherencemodel import CoherenceModel

def compute_coherence_values(dictionary, corpus, path, limit, start=2, step=3, passes=5):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    path : path to input texts
    limit : Max num of topics
    passes: the number of times the entire lda model & coherence values are calculated

    Returns:
    -------
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    
    coherence_values = []
    
    tokens = list(doc_stream(path))
    
    for iter_ in range(passes):
        for num_topics in range(start, limit, step):
            stream = doc_stream(path)
            model = LdaMulticore(corpus=corpus, num_topics=num_topics, id2word=dictionary, workers=4)
            coherencemodel = CoherenceModel(model=model,dictionary=dictionary,corpus=corpus, coherence='u_mass')
            coherence_values.append({'pass': iter_, 
                                     'num_topics': num_topics, 
                                     'coherence_score': coherencemodel.get_coherence()
                                    })

    return coherence_values

In [ ]:
# Can take a long time to run.
coherence_values = compute_coherence_values(dictionary=id2word, 
                                                        corpus=corpus, 
                                                        path=path, 
                                                        start=2, 
                                                        limit=40, 
                                                        step=6,
                                                        passes=40)

In [ ]:
topic_coherence = pd.DataFrame.from_records(coherence_values)

In [ ]:
topic_coherence.head()

In [ ]:
import seaborn as sns

ax = sns.lineplot(x="num_topics", y="coherence_score", data=topic_coherence)

In [ ]:
# Print the coherence scores
for m, cv in zip(x, coherence_values):
    print("Num Topics =", m, " has Coherence Value of", round(cv, 4))

In [ ]:
lda[id2word.doc2bow(tokenize("This is a sample document to score with a topic distribution."))]

## Challenge
### *Can we see if one of the authors focus more on men than women?*

*  Use Spacy for text prepocessing
*  Extract the Named Entities from the documents using Spacy (command is fairly straight forward)
*  Create unique list of names from the authors (you'll find that there are different types of named entities not all people)
*  Label the names with genders (can you this by hand or you use the US census name lists)
*  Customize your processing to replace the proper name with your gender from the previous step's lookup table
*  Then follow the rest of the LDA flow

In [ ]:
import spacy

nlp = spacy.load('en_core_web_lg')

In [ ]:
test = "Ned asked me a question about England today."

In [ ]:
doc = nlp(test)

for token in doc:
    print(token.text, token.lemma_, token.pos_)

In [ ]:
for ent in doc.ents:
    print(ent.text, ent.label_)

In [ ]:
def doc_stream(path):
    for f in os.listdir(path):
        with open(os.path.join(path,f)) as t:
            text = t.read().strip('\n')
            yield text

def get_people(docstream):
    
    ppl = []
    
    for d in docstream:
        
        doc = nlp(d)
        
        for ent in doc.ents:
            
            if ent.label_ == "PERSON":
                ppl.append(ent.lemma_)
                
    return set(ppl)

In [ ]:
people = get_people(doc_stream(path))

In [ ]:
doc = nlp(next(doc_stream(path)))

In [ ]:
doc.ents[0].lemma_

# Sources

### *References*
* [Andrew Ng et al paper on LDA](https://ai.stanford.edu/~ang/papers/jair03-lda.pdf)
* On [Coherence](https://pdfs.semanticscholar.org/1521/8d9c029cbb903ae7c729b2c644c24994c201.pdf)

### *Resources*

* [Gensim](https://radimrehurek.com/gensim/): Python package for topic modeling, nlp, word vectorization, and few other things. Well maintained and well documented.
* [Topic Modeling with Gensim](http://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/#11createthedictionaryandcorpusneededfortopicmodeling): A kind of cookbook for LDA with gensim. Excellent overview, but the you need to be aware of missing import statements and assumed prior knowledge.
* [Chinese Restuarant Process](https://en.wikipedia.org/wiki/Chinese_restaurant_process): That really obscure stats thing I mentioned... 
* [PyLDAvis](https://github.com/bmabey/pyLDAvis): Library for visualizing the topic model and performing some exploratory work. Works well. Has a direct parrell implementation in R as well. 
* [Rare Technologies](https://rare-technologies.com/): The people that made & maintain gensim and a few other libraries.
* [Jane Austen v. Charlotte Bronte](https://www.literaryladiesguide.com/literary-musings/jane-austen-charlotte-bronte-different-alike/)